# Konvoluční síť pro klasifikaci obrázků

Cílem tohoto cvičení je vytvoření jednoduché konvoluční sítě pro klasifikaci obrázků do 10 tříd.

## Dataset

Jedná se o datovou sadu 50 000 barevných tréninkových obrázků 32x32 a 10 000 testovacích obrázků označených v 10 kategoriích. 

Další informace naleznete na https://www.cs.toronto.edu/~kriz/cifar.html

## Třídy
- 0 	airplane
- 1 	automobile
- 2 	bird
- 3 	cat
- 4 	deer
- 5 	dog
- 6 	frog
- 7 	horse
- 8 	ship
- 9 	truck


# 1. Načtení datasetu

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
# Načtení dat pomocí vestavěné funkce
from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [ ]:
# Nastavení pojmenování tříd podle pořadí
class_names=["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
num_classes = len(class_names)

# 2. Průzkum dat

In [ ]:
# Velikosti numpy polí
assert X_train.shape == (50000, 32, 32, 3)
assert X_test.shape == (10000, 32, 32, 3)
assert Y_train.shape == (50000, 1)
assert Y_test.shape == (10000, 1)

print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

In [ ]:
# Obrázek 32x32 pixelů
# 3 barvy
# Hodnoty 0 - 255
# Datový typ - uint8
X_train[0]

In [ ]:
# Kontrola výstupních dat
print(np.unique(Y_train))
print(np.unique(Y_test))

In [ ]:
# Četnost tříd
sns.displot(Y_train)

In [ ]:
sns.displot(Y_test)

# 3. Vizualizace dat

In [ ]:
def show_images (images, labels, rows=6, cols=10):
    """
     Metoda pro zobrazení náhledu obrázků
    """
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows))
    plt.subplots_adjust(bottom=0)
    
    for idx in range (0, rows * cols):
        ridx=idx//cols
        cidx=idx % cols        
        ax= axes[ridx, cidx]
        ax.axis("off")
        ax.imshow(images[idx])
        label = class_names[labels[idx][0]]
        ax.set_title(f"{label}")        

In [ ]:
show_images(X_train, Y_train, 3, 10)

# 4. Příprava dat

In [ ]:
# Převedení dat z int na float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
 
# Standardizace z rozsahu 0 - 255 na 0 - 1
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
X_train[0]

In [ ]:
# binární encoding kategorií
from keras.utils import to_categorical
num_classes = 10
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

In [ ]:
Y_train[0]

# 5. Neuronová síť

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input

In [ ]:
model = Sequential()

# input_shape - Vstupní rozměry - 32 x 32 x 3
# filters     - 32 konvolučních filtrů
# kernel_size - velikost konvolučních filtrů 3 x 3
# padding     - zarovnání same vede k rovnoměrnému vyplňování vlevo/vpravo nebo nahoru/dolů od vstupu
# activation  - aktivační funkce ReLU
model.add(Input(shape=(32,32,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))

# Maxpooling, velikost pool je 2 x 2
model.add(MaxPooling2D(pool_size=(2,2)))

# Vrstva Dropout náhodně nastavuje vstupní neurony na 0, což pomáhá zabránit nadměrnému přizpůsobení sítě trénovacím datům
# Vstupy, které nejsou nastaveny na 0, se škálují o 1 / (1 - rate) tak, aby se součet všech vstupů nezměnil.
model.add(Dropout(0.3))

# Druhy blok konvolučních vrstev
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

# Třetí blok konvolučních vrstev
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

# Převedení více rozměrné mapy příznaků na 1 rozměrový vektor
model.add(Flatten())

# Klasifikační část s plně propojenými vrstvami s aktivačními funkce relu a softmax
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))    # num_classes = 10

In [ ]:
model.summary()

In [ ]:
from keras_visualizer import visualizer 
visualizer(model, file_format='png', view=True)

In [ ]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

# 6. Učení

In [ ]:
# zastavení učení, když se val_loss nebude příliš měnit
early_stop = keras.callbacks.EarlyStopping(monitor='accuracy', patience=30)

In [ ]:
# trénování modelu
history = model.fit(X_train, Y_train, batch_size=64, epochs=15, callbacks=[early_stop])

In [ ]:
# Uložení modelu
model.save('classification_model_cifar10.keras')

# 7. Historie učení

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show()

# 8. Ověření modelu

In [ ]:
model = keras.models.load_model('classification_model_cifar10.keras')

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred[0]

In [ ]:
Y_test[0]

In [ ]:
# Výběr třídy podle nejvýšší pravděpodobnosti
Y_pred_best_answer = np.argmax(Y_pred, axis=-1)
Y_pred_best_answer

In [ ]:
# totéž provedeme s testovacími odpovědmi (realita)
# ideálně se obě pole rovnají
Y_test_best_answer=np.argmax(Y_test, axis=-1)
Y_test_best_answer

In [ ]:
# Vykreslení confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cf_matrix=confusion_matrix(Y_test_best_answer, Y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

In [ ]:
# R2 skóre pro nejlepší odpovědi
from sklearn.metrics import r2_score
r2 = r2_score(Y_test_best_answer, Y_pred_best_answer)
print('R2 score: {}'.format(r2))

In [ ]:
# Ztrátová funkce a přesnost modelu
scores = model.evaluate(X_test, Y_test, verbose=0)
print (f"Loss function: {scores[0]}")
print (f"Accuracy: {scores[1]}")

In [ ]:
# Zobrazení přesnosti pro jednotlivé třídy
class_correct, class_count = [0]*10, [0]*10

for i in range(Y_test.shape[0]):    
    if (Y_test_best_answer[i] == Y_pred_best_answer[i]):
        class_correct[Y_test_best_answer[i]] +=1
    class_count[Y_test_best_answer[i]] += 1
    
for i in range(10):
    print (f"Přesnost pro {class_names[i]}: {class_correct[i]/class_count[i]:.2%}") 

In [ ]:
def show_wrong_predictions(X_test, Y_test, Y_pred, rows=5, cols=5):    
    """ Zobrazení špatných odpovědí """
    idx = 0
    max_examples = rows * cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows), constrained_layout=True)    
    for i in range(Y_test.shape[0]):    
        if (Y_test[i] != Y_pred[i]):                                    
            ridx=idx // cols
            cidx=idx % cols            
            ax = axes[ridx, cidx]
            ax.axis("off")
            ax.imshow(X_test[i].reshape(32,32,3), cmap="gray_r")
            ax.set_title(f"{class_names[Y_test[i]]} != {class_names[Y_pred[i]]}")
            idx +=1
            if (idx == max_examples):
                break

In [ ]:
# zobrazení špatných předpovědí
show_wrong_predictions(X_test, Y_test_best_answer, Y_pred_best_answer, 2, 10)